In [83]:
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()


'/Users/csr/.wdm/drivers/chromedriver/mac64/124.0.6367.60/chromedriver-mac-x64/chromedriver'

In [84]:
from selenium import webdriver
browser = webdriver.Chrome()

In [85]:
from selenium.webdriver.common.by import By
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [69]:
# 1페이지 링크 데이터 전부 수집
##한개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
for i in datas:
    print(i.get_attribute('href'))
#<a class="gd_name" href="/Product/Goods/124999476" onclick="wiseLogV2('BS', '001_005_001', ''); ">불변의 법칙</a>
#<a class="gd_name" href="/Product/Goods/126010792" onclick="wiseLogV2('BS', '001_005_001', ''); ">나는 읽고 쓰고 버린다</a>

https://www.yes24.com/Product/Goods/126010792
https://www.yes24.com/Product/Goods/124999476
https://www.yes24.com/Product/Goods/126029503
https://www.yes24.com/Product/Goods/125295101
https://www.yes24.com/Product/Goods/125992096
https://www.yes24.com/Product/Goods/125491624
https://www.yes24.com/Product/Goods/125651056
https://www.yes24.com/Product/Goods/126110769
https://www.yes24.com/Product/Goods/117014613
https://www.yes24.com/Product/Goods/125576919
https://www.yes24.com/Product/Goods/123675187
https://www.yes24.com/Product/Goods/122120495
https://www.yes24.com/Product/Goods/126029190
https://www.yes24.com/Product/Goods/125973831
https://www.yes24.com/Product/Goods/90926731
https://www.yes24.com/Product/Goods/123400303
https://www.yes24.com/Product/Goods/96547408
https://www.yes24.com/Product/Goods/126012006
https://www.yes24.com/Product/Goods/125557465
https://www.yes24.com/Product/Goods/123930880
https://www.yes24.com/Product/Goods/124043812
https://www.yes24.com/Product/Goods/

In [70]:
# 3페이지까지의 링크 데이터 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
import time

link_list = []
for i in range(1,4):
    print("*" * 10, f"현재 {i} 페이지 수집 중 입니다.","*" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    ### 1페이지 전체의 링크 데이터
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href') # element 요소
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # elements : 리스트
    
    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
        
    time.sleep(3)
    
print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
********** 현재 2 페이지 수집 중 입니다. **********
********** 현재 3 페이지 수집 중 입니다. **********
['https://www.yes24.com/Product/Goods/126010792', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/126029503', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/125992096', 'https://www.yes24.com/Product/Goods/125491624', 'https://www.yes24.com/Product/Goods/125651056', 'https://www.yes24.com/Product/Goods/126110769', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/125576919', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/126029190', 'https://www.yes24.com/Product/Goods/125973831', 'https://www.yes24.com/Product/Goods/90926731', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/96547408', 'https://www.yes24.com/Product/Goods/12601200

In [8]:
len(link_list)

72

In [86]:

# 상세 페이지로 이동
# for link in link_list:
# #     # 상세 페이지로 이동
#      browser.get(link)
    
browser.get(link_list[13])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]
title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks

('마이크로 리추얼 : 사소한 것들의 힘',
 '장재열 저',
 '한국경제신문',
 '2024년 04월 25일',
 '10.0',
 '15',
 '40,194',
 '18,000',
 '베스트 국내도서 14위',
 '국내도서 top20 1주')

In [76]:
text = "'베스트 국내도서 1위 | 국내도서 1위 6주'"

text.split(" | ")[0]

["'베스트", '국내도서', '1위']

In [87]:
# 데이터베이스 연동 후 -> 수정된 데이터를 DB에 저장
import pymysql #pip install pymysql
import time
import re
from datetime import datetime
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '4682',
    db = 'yes24',
    charset = 'utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)
        
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        
        #2023년 12월 25일 -> 2023-12-25
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day)) 
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"
        
        rating = browser.find_element(By.CLASS_NAME, 'yes_b').text
        if len(rating) > 0:
            rating = float(rating)
        else:
            rating = None
        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text.replace(",","")
        del match
        match = re.search(r"\d+", review)
        if match:
            review = int(match.group())
        else:
            review = None
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",", ""))
        
        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",", ""))
        
        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        print('full_text : ' , full_text) # 베스트만 들어옴
        parts = full_text.split(" | ")
        if len(parts) == 1:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking_part = parts[0]
                ranking = ''.join(filter(str.isdigit, ranking_part)) #숫자만 추출
            except:
                ranking = 0
                
            try:
                ranking_weeks_part = parts[1]
                ranking_weeks = ''.join(filter(str.isdigit, ranking_weeks_part.split()[-1]))
            except:
                ranking_weeks = 0
                                    
        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES(
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s 
            )
            """
    
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()
        time.sleep(0.1)

full_text :  베스트 국내도서 1위 | 국내도서 top20 1주
full_text :  베스트 국내도서 2위 | 국내도서 1위 6주
full_text :  베스트 국내도서 3위 | 국내도서 top20 1주
full_text :  베스트 국내도서 4위 | 국내도서 top20 5주
full_text :  베스트 국내도서 5위 | 국내도서 top20 1주
full_text :  베스트 국내도서 6위 | 국내도서 top20 5주
full_text :  베스트 국내도서 7위 | 국내도서 top20 1주
full_text :  베스트 국내도서 8위
full_text :  베스트 국내도서 9위 | 국내도서 1위 22주
full_text :  베스트 국내도서 10위 | 국내도서 top20 3주
full_text :  베스트 국내도서 11위 | 국내도서 top20 17주
full_text :  베스트 국내도서 12위 | 국내도서 1위 12주
full_text :  베스트 국내도서 13위 | 국내도서 top20 1주


DataError: (1366, "Incorrect integer value: '예약판매' for column 'review' at row 1")

In [23]:
# 상세 페이지 이동 후 데이터 크롤링
title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLA3SS_NAME, 'yes_b').text
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[1][:-1]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1].split(" ")[2][:-1]

title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks

('불변의 법칙',
 '모건 하우절 저/이수경 역',
 '서삼독',
 '2024년 02월 28일',
 '9.7',
 '142',
 '499,809',
 '25,000',
 '1',
 '6')